In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image

In [2]:
# Loaded a set of images
root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

In [3]:
files = os.listdir(image_dir)

In [4]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

In [5]:
imgs = [load_image(image_dir + files[i]) for i in range(len(files))]
print(files[0])

gt_imgs = [load_image(gt_dir + files[i]) for i in range(len(files))]
print(files[0])

# n = 10 # Only use 10 images for training

satImage_001.png
satImage_001.png


In [6]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

In [7]:
def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

In [8]:
imgs[0].shape

(400, 400, 3)

In [9]:
img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(len(imgs))]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(len(gt_imgs))]

In [11]:
len(img_patches[0])

625

In [12]:
# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [17]:
def build_k_indices(N, k_fold, seed):
    interval = int(N/k_fold)
    np.random.seed(seed)
    
    indices = np.random.permutation(N)

    k_indices = None
    if N%k_fold == 0:
        k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    else:
        k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold-1)]
        k_indices.append(indices[(k_fold-1)*interval : N])
    return np.array(k_indices)

In [13]:
from sklearn import linear_model

In [25]:
foreground_threshold = 0.25

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

In [21]:
# Extract 6-dimensional features consisting of average RGB color as well as variance
def extract_features(img):
    feat_m = np.mean(img, axis=(0,1))
    feat_v = np.var(img, axis=(0,1))
    feat = np.append(feat_m, feat_v)
    return feat

# Extract 2-dimensional features consisting of average gray color as well as variance
def extract_features_2d(img):
    feat_m = np.mean(img)
    feat_v = np.var(img)
    feat = np.append(feat_m, feat_v)
    return feat

# Extract features for a given image
def extract_img_features(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
    return X

In [30]:
_X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
_Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [43]:
_X[0]

array([ 0.19116116,  0.01471492], dtype=float32)

In [31]:
from sklearn.metrics import recall_score, precision_score

In [98]:
def cross_validation(X, Y, k_indices, k
                     , model = linear_model.LogisticRegression(C=1e5, class_weight="balanced")):
    
    testing_im = X[k_indices[k]].tolist()
    testing_gt = Y[k_indices[k]].tolist()
    
    training_im = []
    training_gt = []
    for j in range(len(k_indices)):
        if j != k:
            training_im += X[k_indices[j]].tolist()
            training_gt += Y[k_indices[j]].tolist()

    model.fit(training_im, training_gt)
    prediction = model.predict(testing_im)
    
    #TODO: refactor
    testing_gt = np.array(testing_gt)
    fp = calc_fp(prediction, testing_gt)
    fn = calc_fn(prediction, testing_gt)
    tp = calc_tp(prediction, testing_gt)
    
    print(fp, tp, fn)
    r = recall(tp, fn)
    p = precision(tp, fp)
    score = f_mean(p, r)
    
    return r, p, score

In [88]:
def cross_validation_demo():
    seed = 1
    k_fold = 4
    
    k_indices = build_k_indices(img_patches.shape[0], k_fold, seed)

    res = []
    for k in range(k_fold):
        res.append(cross_validation(_X, _Y, k_indices, k))
    return res

In [102]:
Z = cross_validation_demo()
Z

5755 2793 1284
5783 2803 1250
5789 2757 1311
5671 2778 1215


[(0.6850625459896983, 0.32674309780065514, 0.4424554455445545),
 (0.6915864791512459, 0.3264616818075938, 0.44354774903077776),
 (0.6777286135693216, 0.32260706763398084, 0.4371333439035992),
 (0.695717505634861, 0.32879630725529646, 0.4465520012859669)]

In [84]:
def calc_tp(pred, true):
    p1 = np.where(pred == 1)
    t1 = np.where(true == 1)
    
    tp = set(p1[0].tolist()) & set(t1[0].tolist())
    return len(tp)

In [100]:
def calc_fn(pred, true):
    p0 = np.where(pred == 0)
    t1 = np.where(true == 1)
    
    fn = set(p0[0].tolist()) & set(t1[0].tolist())
    return len(fn)

In [77]:
recall = lambda tp, fn: tp/(tp + fn)
precision = lambda tp, fp: tp/(tp + fp)
f_mean = lambda p, r: 2*p*r/(p+r)

In [101]:
def calc_fp(pred, true):
    p1 = np.where(pred == 1)
    t0 = np.where(true == 0)
    
    fp = set(p1[0].tolist()) & set(t0[0].tolist())
    return len(fp)